In [1]:
import os
from dotenv import load_dotenv
import pinecone
from llama_index.vector_stores import PineconeVectorStore
import fitz

load_dotenv(dotenv_path='.env')
api_key = os.environ["PINECONE_API_KEY"]
environment = os.environ["PINECONE_ENVIRONMENT"]
pinecone.init(api_key=api_key, environment=environment)

/home/phinguyen/projects/RAG/venv/lib/python3.12/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
index_name = "llamaindex-rag-fs"
pinecone.delete_index(index_name)
pinecone.create_index(
    index_name, dimension=1536, metric='euclidean', pod_type='p1'
)

In [5]:
pinecone_index = pinecone.Index(index_name=index_name)
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)

In [7]:
file_path = "./data/llama2.pdf"
doc = fitz.open(file_path)

In [9]:
type(doc)

fitz.fitz.Document

In [11]:
for doc_idx, page in enumerate(doc):
    print(doc_idx)
    print(page)

0
page 0 of ./data/llama2.pdf
1
page 1 of ./data/llama2.pdf
2
page 2 of ./data/llama2.pdf
3
page 3 of ./data/llama2.pdf
4
page 4 of ./data/llama2.pdf
5
page 5 of ./data/llama2.pdf
6
page 6 of ./data/llama2.pdf
7
page 7 of ./data/llama2.pdf
8
page 8 of ./data/llama2.pdf
9
page 9 of ./data/llama2.pdf
10
page 10 of ./data/llama2.pdf
11
page 11 of ./data/llama2.pdf
12
page 12 of ./data/llama2.pdf
13
page 13 of ./data/llama2.pdf
14
page 14 of ./data/llama2.pdf
15
page 15 of ./data/llama2.pdf
16
page 16 of ./data/llama2.pdf
17
page 17 of ./data/llama2.pdf
18
page 18 of ./data/llama2.pdf
19
page 19 of ./data/llama2.pdf
20
page 20 of ./data/llama2.pdf
21
page 21 of ./data/llama2.pdf
22
page 22 of ./data/llama2.pdf
23
page 23 of ./data/llama2.pdf
24
page 24 of ./data/llama2.pdf
25
page 25 of ./data/llama2.pdf
26
page 26 of ./data/llama2.pdf
27
page 27 of ./data/llama2.pdf
28
page 28 of ./data/llama2.pdf
29
page 29 of ./data/llama2.pdf
30
page 30 of ./data/llama2.pdf
31
page 31 of ./data/llama2.

In [12]:
from datetime import datetime
last = datetime.now()

In [14]:
type(last)

datetime.datetime

In [13]:
current = datetime.now()
delta = current - last
print(delta)

0:00:34.035464


In [15]:
def time_diff(last:datetime, now:datetime) -> str:
    delta = now - last
    # Calculate hours, minutes, and seconds
    hours, remainder = divmod(delta.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours}h{minutes}m{seconds}s"

In [16]:
print(time_diff(last=last, now=datetime.now()))

0h4m30s


In [19]:
# 2. Use a Text Splitter to Split Documents
from llama_index.text_splitter import SentenceSplitter

text_splitter = SentenceSplitter(
    chunk_size=1024
)

text_chunks = []
doc_idxs = []
for doc_idx, page in enumerate(doc):
    print(f'Start process page {doc_idx + 1}')
    start_time = datetime.now()
    
    page_text = page.get_text("text")
    cur_text_chunk = text_splitter.split_text(page_text)
    text_chunks.extend(cur_text_chunk)
    doc_idxs.extend([doc_idx] * len(cur_text_chunk))
    
    print(f'Finish process page {doc_idx + 1}')
    print(f'It takes {time_diff(start_time, datetime.now())} to process page {doc_idx + 1}')

[nltk_data] Downloading package punkt to /tmp/llama_index...


KeyboardInterrupt: 